# Test

In [7]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# API 요청 URL
url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
query = '예래해안도로'
params = {'query': query}

# API Key 정보
headers = {
    'x-ncp-apigw-api-key-id': os.getenv('X-NCP-APIGW-API-KEY-ID'),
    'x-ncp-apigw-api-key': os.getenv('X-NCP-APIGW-API-KEY'),
    'Accept': 'application/json'
}

# GET 요청 보내기
response = requests.get(url, headers=headers, params=params)

# 응답 결과 출력
if response.status_code == 200:
    print(response.json())  # JSON 응답 출력
else:
    print(f"Error: {response.status_code}")


{'status': 'OK', 'meta': {'totalCount': 0, 'count': 0}, 'addresses': [], 'errorMessage': ''}


In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# API URL 및 요청 URL
url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"

# 요청에 필요한 파라미터
params = {
    'coords': '127.585,34.9765',
    'output': 'json',
    'orders': 'legalcode,admcode,addr,roadaddr'
}

# API Key 정보
headers = {
    'x-ncp-apigw-api-key-id': os.getenv('X-NCP-APIGW-API-KEY-ID'),
    'x-ncp-apigw-api-key': os.getenv('X-NCP-APIGW-API-KEY'),
    'Accept': 'application/json'
}

# GET 요청 보내기
response = requests.get(url, headers=headers, params=params)

# 응답 결과 출력
if response.status_code == 200:
    print(response.json())  # JSON 응답 출력
else:
    print(f"Error: {response.status_code}")

{'status': {'code': 0, 'name': 'ok', 'message': 'done'}, 'results': [{'name': 'legalcode', 'code': {'id': '4623025024', 'type': 'L', 'mappingId': '1223025024'}, 'region': {'area0': {'name': 'kr', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}, 'area1': {'name': '전라남도', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 126.957167, 'y': 34.900727}}, 'alias': '전남'}, 'area2': {'name': '광양시', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.695899, 'y': 34.940658}}}, 'area3': {'name': '광양읍', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.58105, 'y': 34.97341}}}, 'area4': {'name': '읍내리', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.5863, 'y': 34.9751}}}}}, {'name': 'admcode', 'code': {'id': '4623025000', 'type': 'L', 'mappingId': '12230250'}, 'region': {'area0': {'name': 'kr', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}, 'area1': {'name': '전라남도', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 126.957167, 'y': 34.900727}}, 'alias': '전남'}, 'area2': {'name': '광양시', '

# 관광지 데이터 정제

In [1]:
import pandas as pd

# CSV 파일 읽기 (인코딩 변경)
try:
    df = pd.read_csv('/media/jys/ssd/workspace/003_Competition/006_KAIT-2024-Big-Contest/old_addr_real_final.csv', encoding='euc-kr')
except UnicodeDecodeError:
    df = pd.read_csv('/media/jys/ssd/workspace/003_Competition/006_KAIT-2024-Big-Contest/old_addr_real_final.csv', encoding='utf-8')

# Unnamed 컬럼 제거
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 0번 컬럼 제거
df = df.iloc[:, 1:]

# 변경된 데이터프레임을 CSV 파일로 저장
df.to_csv('/media/jys/ssd/workspace/003_Competition/006_KAIT-2024-Big-Contest/old_addr_real_final_cleaned.csv', index=False, encoding='utf-8-sig')

print("Unnamed 컬럼과 0번 컬럼이 제거되고 새로운 CSV 파일로 저장되었습니다.")

Unnamed 컬럼과 0번 컬럼이 제거되고 새로운 CSV 파일로 저장되었습니다.


# 관광지 네이버 지도 위경도 API 호출

In [2]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# .env 파일 로드
load_dotenv()

# API 키 및 URL 설정
GEOCODE_URL = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
HEADERS = {
    'X-NCP-APIGW-API-KEY-ID': os.getenv('X-NCP-APIGW-API-KEY-ID'),
    'X-NCP-APIGW-API-KEY': os.getenv('X-NCP-APIGW-API-KEY'),
    'Accept': 'application/json'
}

def geocode(query):
    params = {'query': query}
    response = requests.get(GEOCODE_URL, headers=HEADERS, params=params)
    if response.status_code == 200:
        result = response.json()
        if result['addresses']:
            return result['addresses'][0]['y'], result['addresses'][0]['x']
    return None, None

def process_row(row):
    lat, lng = geocode(row['New_ADDR'])
    time.sleep(0.1)  # API 호출 제한을 고려한 딜레이
    return lat, lng

# CSV 파일 읽기
df = pd.read_csv('/media/jys/ssd/workspace/003_Competition/006_KAIT-2024-Big-Contest/old_addr_real_final_cleaned.csv')

# 새로운 컬럼 추가
df['lat'] = None
df['lng'] = None

# ThreadPoolExecutor를 사용한 병렬 처리
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_index = {executor.submit(process_row, row): index for index, row in df.iterrows()}
    
    for future in tqdm(as_completed(future_to_index), total=len(future_to_index)):
        index = future_to_index[future]
        lat, lng = future.result()
        df.at[index, 'lat'] = lat
        df.at[index, 'lng'] = lng

# 결과를 CSV 파일로 저장
df.to_csv('/media/jys/ssd/workspace/003_Competition/006_KAIT-2024-Big-Contest/old_addr_with_coordinates.csv', index=False, encoding='utf-8-sig')

print("위경도 좌표가 추가된 새로운 CSV 파일이 저장되었습니다.")


100%|██████████| 1353/1353 [00:57<00:00, 23.45it/s]

위경도 좌표가 추가된 새로운 CSV 파일이 저장되었습니다.
